In [0]:
'''
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
import argparse
from tensorflow.keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, Lambda, LSTM, TimeDistributed, Masking, Bidirectional
from tensorflow.keras.layers import Reshape, Flatten, Dropout, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import os, pickle
import numpy as np
'''
import numpy as np
import pandas as pd
import pickle
import os, sys
from collections import Counter, defaultdict
from functools import cmp_to_key
import argparse
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, Lambda, LSTM, TimeDistributed, Masking, Bidirectional, concatenate, Layer
from keras.layers import Reshape, Flatten, Dropout, Concatenate, Dot
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model, load_model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import keras
#from keras_multi_head import MultiHead
#from keras_multi_head import MultiHeadAttention
from datetime import datetime
#from keras_self_attention import SeqSelfAttention
import tensorflow as tf
from keras import regularizers

x = pickle.load(open("/content/drive/My Drive/mca/MCA_Project/data/pickles/data_{}.p".format("emotion"),"rb"))
revs, W, word_idx_map, vocab, _, label_index = x[0], x[1], x[2], x[3], x[4], x[5]

def get_word_indices(data_x):
  length = len(data_x.split())
  return np.array([word_idx_map[word] for word in data_x.split()] + [0]*(50-length))[:50]

def comp_id(x, y):
  xd = int(x[:x.find('_')])
  xu = int(x[x.find('_')+1:])

  yd = int(y[:y.find('_')])
  yu = int(y[y.find('_')+1:])

  if xd != yd:
    return xd - yd
  else:
    return xu - yu


def preprocess():

  train_data, val_data, test_data = {},{},{}

  counts_train = np.zeros((5,1))
  counts_test = np.zeros((5,1))
  counts_val = np.zeros((5,1))

  tr_labels, v_labels, ts_labels = {}, {}, {}

  for i in range(len(revs)):

    utterance_id = revs[i]['dialog']+"_"+revs[i]['utterance']
    
    sentence_word_indices = get_word_indices(revs[i]['text'])
    
    label = label_index[revs[i]['y']]

    if label != 0 and label != 3 and label != 4 and label != 6:
      continue

    if label == 0:
      label = 0
    elif label == 3:
      label = 1
    elif label == 4:
      label = 2
    else:
      label = 3 

    if revs[i]['split']=="train":
      tr_labels[utterance_id] = label
    if revs[i]['split']=="val":
      v_labels[utterance_id] = label
    if revs[i]['split']=="test":
      ts_labels[utterance_id] = label

    if revs[i]['split']=="train" and counts_train[label] > 683:
      continue

    if revs[i]['split']=="train":
        train_data[utterance_id]=(sentence_word_indices,label)
        counts_train[label] += 1
    elif revs[i]['split']=="val":
        val_data[utterance_id]=(sentence_word_indices,label)
        counts_val[label] += 1
    elif revs[i]['split']=="test":
        test_data[utterance_id]=(sentence_word_indices,label)
        counts_test[label] += 1

  dialogs = []
  utrs = -1
  d_cur = -1

  t_d = {}
  t_map = {}
  sorted_tr_keys = sorted(train_data.keys(), key=cmp_to_key(comp_id))

  for i in sorted_tr_keys:
    d = i[:i.find('_')]
    u = i[i.find('_') + 1:]
    ouid = d + '_' + u

    if d not in dialogs:
      d_cur += 1
      utrs = 0
      dialogs.append(d)
    else:
      utrs += 1

    df = d_cur
    uf = utrs

    uid = str(df) +'_' + str(uf)
    t_d[uid] = train_data[i]

    t_map[uid] = ouid

  dialogs = []
  utrs = -1
  d_cur = -1

  v_d = {}
  v_map = {}
  sorted_val_keys = sorted(val_data.keys(), key=cmp_to_key(comp_id))

  for i in sorted_val_keys:
    d = i[:i.find('_')]
    u = i[i.find('_') + 1:]
    ouid = d + '_' + u

    if d not in dialogs:
      d_cur += 1
      utrs = 0
      dialogs.append(d)
    else:
      utrs += 1

    df = d_cur
    uf = utrs

    uid = str(df) +'_' + str(uf)
    v_d[uid] = val_data[i]
    v_map[uid] = ouid

  dialogs = []
  utrs = -1
  d_cur = -1

  ts_d = {}
  ts_map = {}
  sorted_ts_keys = sorted(test_data.keys(), key=cmp_to_key(comp_id))

  for i in sorted_ts_keys:
    d = i[:i.find('_')]
    u = i[i.find('_') + 1:]
    ouid = d + '_' + u

    if d not in dialogs:
      d_cur += 1
      utrs = 0
      dialogs.append(d)
    else:
      utrs += 1

    df = d_cur
    uf = utrs

    uid = str(df) +'_' + str(uf)
    ts_d[uid] = test_data[i]
    ts_map[uid] = ouid
  
  return t_d, v_d, ts_d, t_map, v_map, ts_map, tr_labels, v_labels, ts_labels


#preprocess()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

max_length=50 # Maximum length of the sentence

class Dataloader:
    
    def __init__(self, mode=None):

        self.MODE = mode # Sentiment or Emotion classification mode
        self.max_l = max_length


        x = pickle.load(open("/content/drive/My Drive/mca/MCA_Project/data/pickles/data_{}.p".format(self.MODE.lower()),"rb"))
        self.revs, self.W, self.word_idx_map, self.vocab, _, label_index = x[0], x[1], x[2], x[3], x[4], x[5]
        
        self.num_classes = 4
        print("Labels used for this classification: {'neutral': 0, 'sadness': 1, 'joy': 2, 'anger': 3}")

        self.train_data, self.val_data, self.test_data, self.tr_map, self.v_map, self.ts_map, self.tr_labels, self.v_labels, self.ts_labels = preprocess()

        # Creating dialogue:[utterance_1, utterance_2, ...] ids
        self.train_dialogue_ids = self.get_dialogue_ids(self.train_data.keys())
        self.val_dialogue_ids = self.get_dialogue_ids(self.val_data.keys())
        self.test_dialogue_ids = self.get_dialogue_ids(self.test_data.keys())

        # Max utternance in a dialog in the dataset
        self.max_utts = self.get_max_utts(self.train_dialogue_ids, self.val_dialogue_ids, self.test_dialogue_ids)

    def get_dialogue_ids(self, keys):
        ids=defaultdict(list)
        for key in keys:
            ids[key.split("_")[0]].append(int(key.split("_")[1]))
        for ID, utts in ids.items():
            ids[ID]=[str(utt) for utt in sorted(utts)]
        return ids

    def get_max_utts(self, train_ids, val_ids, test_ids):
        max_utts_train = max([len(train_ids[vid]) for vid in train_ids.keys()])
        max_utts_val = max([len(val_ids[vid]) for vid in val_ids.keys()])
        max_utts_test = max([len(test_ids[vid]) for vid in test_ids.keys()])
        return np.max([max_utts_train, max_utts_val, max_utts_test])

    def get_one_hot(self, label):
        label_arr = [0]*self.num_classes
        label_arr[label]=1
        return label_arr[:]

    def get_dialogue_text_embs(self):
        key = list(self.train_data.keys())[0]
        
        pad = [0]*len(self.train_data[key][0])

        def get_emb(dialogue_id, local_data):
            dialogue_text = []
            for vid in dialogue_id.keys():
                local_text = []
                for utt in dialogue_id[vid]:
                    local_text.append(local_data[vid+"_"+str(utt)][0][:])
                for _ in range(self.max_utts-len(local_text)):
                    local_text.append(pad[:])
                dialogue_text.append(local_text[:self.max_utts])
            return np.array(dialogue_text)

        self.train_dialogue_features = get_emb(self.train_dialogue_ids, self.train_data)
        self.val_dialogue_features = get_emb(self.val_dialogue_ids, self.val_data)
        self.test_dialogue_features = get_emb(self.test_dialogue_ids, self.test_data)

    def get_dialogue_labels(self):

        def get_labels(ids, data):
            dialogue_label=[]

            for vid, utts in ids.items():
                local_labels=[]
                for utt in utts:
                    local_labels.append(self.get_one_hot(data[vid+"_"+str(utt)][1]))
                for _ in range(self.max_utts-len(local_labels)):
                    local_labels.append(self.get_one_hot(1)) # Dummy label
                dialogue_label.append(local_labels[:self.max_utts])
            return np.array(dialogue_label)

        self.train_dialogue_label=get_labels(self.train_dialogue_ids, self.train_data)
        self.val_dialogue_label=get_labels(self.val_dialogue_ids, self.val_data)
        self.test_dialogue_label=get_labels(self.test_dialogue_ids, self.test_data)

    def get_dialogue_labels_audio(self):

        def get_labels(ids, data, Map, map1):
            dialogue_label=[]

            for vid, utts in ids.items():
                local_labels=[]
                for utt in utts:
                    local_labels.append(self.get_one_hot(Map[map1[vid+'_'+str(utt)]]))
                for _ in range(self.max_utts-len(local_labels)):
                    local_labels.append(self.get_one_hot(1)) # Dummy label
                dialogue_label.append(local_labels[:self.max_utts])
            return np.array(dialogue_label)

        self.train_dialogue_label=get_labels(self.train_dialogue_ids, self.train_data, self.tr_labels, self.tr_map)
        self.val_dialogue_label=get_labels(self.val_dialogue_ids, self.val_data, self.v_labels, self.v_map)
        self.test_dialogue_label=get_labels(self.test_dialogue_ids, self.test_data, self.ts_labels, self.ts_map)

        
    def get_dialogue_lengths(self):

        self.train_dialogue_length, self.val_dialogue_length, self.test_dialogue_length=[], [], []
        for vid, utts in self.train_dialogue_ids.items():
            self.train_dialogue_length.append(len(utts))
        for vid, utts in self.val_dialogue_ids.items():
            self.val_dialogue_length.append(len(utts))
        for vid, utts in self.test_dialogue_ids.items():
            self.test_dialogue_length.append(len(utts))

    def get_masks(self):

        self.train_mask = np.zeros((len(self.train_dialogue_length), self.max_utts), dtype='int')
        for i in range(len(self.train_dialogue_length)):
            self.train_mask[i,:self.train_dialogue_length[i]]=1
        self.val_mask = np.zeros((len(self.val_dialogue_length), self.max_utts), dtype='int')
        for i in range(len(self.val_dialogue_length)):
            self.val_mask[i,:self.val_dialogue_length[i]]=1
        self.test_mask = np.zeros((len(self.test_dialogue_length), self.max_utts), dtype='int')
        for i in range(len(self.test_dialogue_length)):
            self.test_mask[i,:self.test_dialogue_length[i]]=1
        
    def load_text_data(self, ):

        self.get_dialogue_text_embs()
        self.get_dialogue_lengths()
        self.get_dialogue_labels()
        self.get_masks()

    def load_audio_data(self, ):

        AUDIO_PATH = "/content/drive/My Drive/mca/MCA_Project/data/pickles/audio_embeddings_feature_selection_{}.pkl".format(self.MODE.lower())
        self.train_audio_emb, self.val_audio_emb, self.test_audio_emb = pickle.load(open(AUDIO_PATH,"rb"))

        self.get_dialogue_audio_embs()
        self.get_dialogue_lengths()
        self.get_dialogue_labels_audio()
        self.get_masks()

    def get_dialogue_audio_embs(self):
        key = list(self.train_audio_emb.keys())[0]
        pad = [0]*len(self.train_audio_emb[key])

        def get_emb(dialogue_id, audio_emb, Map):
            dialogue_audio=[]
            for vid in dialogue_id.keys():
                local_audio=[]
                for utt in dialogue_id[vid]:
                    try:
                        local_audio.append(audio_emb[Map[vid+"_"+str(utt)]][:])
                    except:
                        print(vid+"_"+str(utt))
                        local_audio.append(pad[:])
                for _ in range(self.max_utts-len(local_audio)):
                    local_audio.append(pad[:])
                dialogue_audio.append(local_audio[:self.max_utts])
            return np.array(dialogue_audio)

        self.train_dialogue_features = get_emb(self.train_dialogue_ids, self.train_audio_emb, self.tr_map)
        self.val_dialogue_features = get_emb(self.val_dialogue_ids, self.val_audio_emb, self.v_map)
        self.test_dialogue_features = get_emb(self.test_dialogue_ids, self.test_audio_emb, self.ts_map)


    def load_video_data(self, ):

        self.val_video_emb = np.load('/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/vgg_x_dev.pkl', allow_pickle='True')
        self.train_video_emb = np.load('/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/vgg_x_train.pkl', allow_pickle='True')
        self.test_video_emb = np.load('/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/vgg_x_test.pkl', allow_pickle='True')
        self.vid_to_row_train = np.load('/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/dia_frames_train_dia_to_row.pkl', allow_pickle=True).item()
        self.vid_to_row_val = np.load('/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/dia_frames_dev_dia_to_row.pkl', allow_pickle=True).item()
        self.vid_to_row_test = np.load('/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/dia_frames_test_dia_to_row.pkl', allow_pickle=True).item()
        
        self.val_video_emb = self.val_video_emb.reshape(self.val_video_emb.shape[0], 2048)
        self.train_video_emb = self.train_video_emb.reshape(self.train_video_emb.shape[0], 2048)
        self.test_video_emb = self.test_video_emb.reshape(self.test_video_emb.shape[0], 2048)
        #print(self.val_video_emb.shape, val_video_emb)
        '''
        from sklearn.preprocessing import StandardScaler
        
        def normalize(emb):
          X = []
          scaler = StandardScaler()

          for e in sorted(emb.keys()):
            X.append(emb[e])

          X = scaler.fit_transform(X)

          for n, e in enumerate(sorted(emb.keys())):
            emb[e] = X[n]

          return emb
          '''

        #self.val_video_emb = normalize(self.val_video_emb)
        #self.test_video_emb = normalize(self.test_video_emb)
        #self.train_video_emb = normalize(self.train_video_emb)
        
        self.get_dialogue_video_embs()
        self.get_dialogue_lengths()
        self.get_dialogue_labels_video()
        self.get_masks()

    def get_dialogue_video_embs(self):
        #key = list(self.train_video_emb.keys())[0]
        pad = [0]*2048
        
        #print(len(pad))
        
        def get_emb(dialogue_id, video_emb, Map, to_row):
            dialogue_video=[]
            ct = 0
            for vid in dialogue_id.keys():
                local_video=[]
                for utt in dialogue_id[vid]:
                    try:
                        #local_video.append(video_emb[Map[vid+"_"+str(utt)]][:])
                        key = Map[vid + "_" + str(utt)]
                        sm = np.zeros((2048,))
                        for k in to_row[key]:
                          tmp = video_emb[k, :]
                          sm += tmp

                        sm = sm/len(to_row[key])

                        sm = list(sm)

                        local_video.append(sm)
                    except:
                        #print(vid+"_"+str(utt), end = " ")
                        local_video.append(pad[:])
                        ct += 1
                    #print(len(local_video))
                for _ in range(self.max_utts-len(local_video)):
                    local_video.append(pad[:])
                dialogue_video.append(local_video[:self.max_utts])
                #print(len(dialogue_video), len(dialogue_video[0]), len(dialogue_video[0][0]))

            #print(np.array(dialogue_video).shape)
            dialogue_video = np.array(dialogue_video)
            dialogue_video = np.nan_to_num(dialogue_video)

            print(np.argwhere(np.isnan(dialogue_video)))
            return dialogue_video

        print("train")
        self.train_dialogue_features = get_emb(self.train_dialogue_ids, self.train_video_emb, self.tr_map, self.vid_to_row_train)
        print("val")
        self.val_dialogue_features = get_emb(self.val_dialogue_ids, self.val_video_emb, self.v_map, self.vid_to_row_val)
        print("test")
        self.test_dialogue_features = get_emb(self.test_dialogue_ids, self.test_video_emb, self.ts_map, self.vid_to_row_test)
        print("")

    def get_dialogue_labels_video(self):

        def get_labels(ids, data, Map, map1):
            dialogue_label=[]

            for vid, utts in ids.items():
                local_labels=[]
                for utt in utts:
                    local_labels.append(self.get_one_hot(Map[map1[vid+'_'+str(utt)]]))
                for _ in range(self.max_utts-len(local_labels)):
                    local_labels.append(self.get_one_hot(1)) # Dummy label
                dialogue_label.append(local_labels[:self.max_utts])
            return np.array(dialogue_label)

        self.train_dialogue_label=get_labels(self.train_dialogue_ids, self.train_data, self.tr_labels, self.tr_map)
        self.val_dialogue_label=get_labels(self.val_dialogue_ids, self.val_data, self.v_labels, self.v_map)
        self.test_dialogue_label=get_labels(self.test_dialogue_ids, self.test_data, self.ts_labels, self.ts_map)




In [0]:
import pickle

class Network1:

	def __init__(self):
		self.classification_mode = "emotion"
		
	def load_data(self,m):
    
		print('Loading data')
    
		self.data = Dataloader(mode = self.classification_mode)
    
		if m == "text":
			self.data.load_text_data()
		elif m == "audio":
			self.data.load_audio_data()
		elif m =="video":
			self.data.load_video_data()
		else:
			exit()
    
		self.train_x = self.data.train_dialogue_features
		self.val_x = self.data.val_dialogue_features
		self.test_x = self.data.test_dialogue_features
    
		self.train_y = self.data.train_dialogue_label
		self.val_y = self.data.val_dialogue_label
		self.test_y = self.data.test_dialogue_label
    
		self.train_mask = self.data.train_mask
		self.val_mask = self.data.val_mask
		self.test_mask = self.data.test_mask
    
		self.train_id = self.data.train_dialogue_ids.keys()
		self.val_id = self.data.val_dialogue_ids.keys()
		self.test_id = self.data.test_dialogue_ids.keys()
    
		self.sequence_length = self.train_x.shape[1]

		self.classes = self.train_y.shape[2]
    
		self.epochs = 15
		self.batch_size = 50

		if m == "text":
			self.train_x_text = self.train_x
			self.val_x_text = self.val_x
			self.test_x_text = self.test_x

			self.train_y_text = self.train_y
			self.val_y_text = self.val_y 
			self.test_y_text = self.test_y 
			
			self.train_mask_text = self.train_mask 
			self.val_mask_text = self.val_mask 
			self.test_mask_text = self.test_mask
			
			self.train_id_text = self.train_id 
			self.val_id_text = self.val_id 
			self.test_id_text = self.test_id 

			self.sequence_length_text = self.sequence_length

		if m == "audio":
			self.train_x_audio = self.train_x
			self.val_x_audio = self.val_x
			self.test_x_audio = self.test_x

			self.train_y_audio = self.train_y
			self.val_y_audio = self.val_y 
			self.test_y_audio = self.test_y 
			
			self.train_mask_audio = self.train_mask 
			self.val_mask_audio = self.val_mask 
			self.test_mask_audio = self.test_mask
			
			self.train_id_audio = self.train_id 
			self.val_id_audio = self.val_id 
			self.test_id_audio = self.test_id 
			self.sequence_length_audio = self.sequence_length

		if m == "video":
			self.train_x_video = self.train_x
			self.val_x_video = self.val_x
			self.test_x_video = self.test_x

			self.train_y_video = self.train_y
			self.val_y_video = self.val_y 
			self.test_y_video = self.test_y 
			
			self.train_mask_video = self.train_mask 
			self.val_mask_video = self.val_mask 
			self.test_mask_video = self.test_mask
			
			self.train_id_video = self.train_id 
			self.val_id_video = self.val_id 
			self.test_id_video = self.test_id 
			self.sequence_length_video = self.sequence_length

	def get_text_lstm(self):
		self.sentence_length = self.train_x.shape[2]
    
		self.embedding_dim = self.data.W.shape[1]
    
		self.vocabulary_size = self.data.W.shape[0]
		
		embedding = Embedding(input_dim=self.vocabulary_size, output_dim=self.embedding_dim, weights=[self.data.W], input_length=self.sentence_length, trainable=False)
    
		def slicer(x, index):
			return x[:,K.constant(index, dtype='int32'),:]
    
		def slicer_output_shape(input_shape):
			shape = list(input_shape)
			assert len(shape) == 3  # batch, seq_len, sent_len
			new_shape = (shape[0], shape[2])
			return new_shape

		def reshaper(x):
			return K.expand_dims(x, axis=3)
    
		def flattener(x):
			x = K.reshape(x, [-1,x.shape[1]*x.shape[2]])
			return x

		def flattener_output_shape(input_shape):
			shape = list(input_shape)
			new_shape = (shape[0], shape[2]*shape[1])
			return new_shape

		inputs = Input(shape=(self.sequence_length, self.sentence_length), dtype='int32')
		
		
		output = []
		for ind in range(self.sequence_length):
			local_input = Lambda(slicer, output_shape=slicer_output_shape, arguments={"index":ind})(inputs) # Batch, word_indices
			#print(K.int_shape(local_input))
			emb_output = embedding(local_input)
			#print(K.int_shape(emb_output))
			reshape = Lambda(reshaper)(emb_output)
			#print(K.int_shape(reshape))
			flatten = Lambda(flattener, output_shape=flattener_output_shape,)(reshape)

			output.append(flatten)

		def stack(x):
			return K.stack(x, axis=1)
      
		outputs = Lambda(stack)(output)
		masked = Masking(mask_value =0)(outputs)
	
		lstm = Bidirectional(LSTM(150, activation='relu', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name = 'lstm_t')(masked)
		
		self.text_lstm_layer = lstm
		
		lstm1 = Bidirectional(LSTM(150, activation='relu', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name="utter_t")(lstm)
		
		#att_text = SeqSelfAttention(attention_activation='sigmoid')(lstm)
		#att_text = AttentionDecoder(100, 300)(lstm)

		output = TimeDistributed(Dense(self.classes,activation='softmax'))(lstm1)

		model = Model(inputs, output)

		model.summary()

		self.text_lstm =  model

		return lstm, inputs

	def get_audio_lstm(self):

		self.embedding_dim = self.train_x.shape[2]

		print("Creating Model...")
		
		inputs = Input(shape=(self.sequence_length, self.embedding_dim), dtype='float32')
		masked = Masking(mask_value =0)(inputs)
		lstm = Bidirectional(LSTM(150, activation='tanh', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name='lstm_a')(masked)
		self.audio_lstm_layer = lstm
		lstm1 = Bidirectional(LSTM(150, activation='tanh', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name="utter_a")(lstm)
		output = TimeDistributed(Dense(self.classes,activation='softmax'))(lstm1)

		model = Model(inputs, output)

		self.audio_lstm = model

		model.summary()

		return lstm, inputs

	def get_video_lstm(self):

		self.embedding_dim = self.train_x.shape[2]

		print("Creating Model...")
		
		inputs = Input(shape=(self.sequence_length, self.embedding_dim), dtype='float32')
		masked = Masking(mask_value =0)(inputs)
		lstm = Bidirectional(LSTM(150, activation='relu', return_sequences = True, dropout=0.2, recurrent_dropout=0.2),name='lstm_a')(masked)
		self.video_lstm_layer = lstm
		lstm1 = Bidirectional(LSTM(150, activation='relu', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name="utter_a")(lstm)
		#output = TimeDistributed(Dense(self.classes,activation='softmax'))(lstm1)
		#lstm2 = Bidirectional(LSTM(100, activation='relu', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name="utter_b")(lstm1)
		output2 = TimeDistributed(Dense(self.classes,activation='softmax'))(lstm1)

		model = Model(inputs, output2)
	
		
		filename = '/content/drive/My Drive/mca/MCA_Project/MELD_Dataset/vgg_balanced_himanshu.sav'
		pickle.dump(model, open(filename, 'wb'))
	
		model.summary()

		self.video_lstm = model

		#model.summary()

		return lstm, inputs

	def get_final_model(self, tl, al, ti, ai):
		ts = Concatenate(axis=-1)([tl, al])
		att_layer = MultiHeadAttention(head_num=3,name='Multi-Head',)([tl, al, ts])
		
		#att_layer = MyAttention()([tl, al])
		
		concat_output = Concatenate(axis=-1, name='concat_layer')([tl, att_layer])
		
		lstm = Bidirectional(LSTM(150, activation='relu', return_sequences = True, dropout=0.2, recurrent_dropout=0.2), name='lstm_f')(concat_output)
		
		output = TimeDistributed(Dense(self.classes,activation='softmax'))(lstm)
	
		self.merged_model = Model([ti, ai], output)
	
	def train_lstm(self, m):
		if m == 'text':
			model = self.text_lstm
		elif m == 'audio':
			model = self.audio_lstm
		elif m == 'video':
			model = self.video_lstm

		
		model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'], sample_weight_mode='temporal')
		early_stopping = EarlyStopping(monitor='val_loss', patience=10)
	
		logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M")
		tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

		summary = model.fit(self.train_x, self.train_y,
		                epochs=self.epochs,
		                batch_size=self.batch_size,
		                sample_weight=self.train_mask,
		                shuffle=True, 
		                callbacks=[early_stopping, tensorboard_callback],
		                validation_data=(self.val_x, self.val_y, self.val_mask))
	
		return summary

	
		#self.test_model(m)
	
	def train_network(self):
		model = self.merged_model
		model.compile(optimizer='adam', loss='categorical_crossentropy', sample_weight_mode='temporal')
		early_stopping = EarlyStopping(monitor='loss', patience=10)
	
		#logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M")
		#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

		model.fit([self.train_x_text, self.train_x_audio], self.train_y,
		                epochs=self.epochs,
		                batch_size=self.batch_size,
		                shuffle=True, 
		                callbacks=[early_stopping, tensorboard_callback],
										sample_weight = self.train_mask,
										validation_data=([self.val_x_text, self.val_x_audio], self.val_y, self.val_mask))
	
		
	def test_model(self, m):
		if m == 'text':
			model = self.text_lstm
			#intermediate_layer_model = Model(input=model.input, output=model.get_layer("lstm_t").output)
		elif m == 'audio':
			model = self.audio_lstm
			#intermediate_layer_model = Model(input=model.input, output=model.get_layer("lstm_a").output)
		elif m == 'merged':
			model = self.merged_model
			#intermediate_layer_model = Model(input=model.input, output=model.get_layer("lstm_f").output)

		'''
		intermediate_output_train = intermediate_layer_model.predict(self.train_x)
		intermediate_output_val = intermediate_layer_model.predict(self.val_x)
		intermediate_output_test = intermediate_layer_model.predict(self.test_x)

		train_emb, val_emb, test_emb = {}, {}, {}
		for idx, ID in enumerate(self.train_id):
		    train_emb[ID] = intermediate_output_train[idx]
		for idx, ID in enumerate(self.val_id):
		    val_emb[ID] = intermediate_output_val[idx]
		for idx, ID in enumerate(self.test_id):
		    test_emb[ID] = intermediate_output_test[idx]
		'''
		calc_test_result(model.predict(self.test_x), self.test_y, self.test_mask)
		#calc_test_result(model.predict(self.train_x), self.train_y, self.train_mask)
		
def calc_test_result(pred_label, test_label, test_mask):

		true_label=[]
		predicted_label=[]

		for i in range(pred_label.shape[0]):
			for j in range(pred_label.shape[1]):
				if test_mask[i,j]==1:
					true_label.append(np.argmax(test_label[i,j] ))
					predicted_label.append(np.argmax(pred_label[i,j] ))
		
	
		print("Confusion Matrix :")
		print(confusion_matrix(true_label, predicted_label))
		print("Classification Report :")
		print(classification_report(true_label, predicted_label, digits=4))
		print('Weighted FScore: \n ', precision_recall_fscore_support(true_label, predicted_label, average='weighted'))
	
	


In [6]:
def plot(summary):
  l = summary.history['loss']
  vl = summary.history['val_loss']

  #a = summary.history['acc']
  #va = summary.history['val_acc']

  import matplotlib.pyplot as plt

  e = len(l)
  iterations = list(range(e))
  plt.plot(iterations, l, color='blue', label='train')
  plt.plot(iterations, vl, color='red', label ='val')
  plt.legend(loc='best')
    
  #axs[1].plot(iterations, a, color='red')
  #axs[1].plot(iterations, va, color='red')



N = Network1()

N.load_data("video")
tl, ti = N.get_video_lstm()
s = N.train_lstm("video")

model = N.video_lstm
calc_test_result(model.predict(N.test_x_video), N.test_y, N.test_mask)
calc_test_result(model.predict(N.train_x_video), N.train_y, N.train_mask)

#N.load_data("text")
#tl, ti = N.get_text_lstm()
#N.train_lstm("text")

#print(N.train_x, N.train_y)

#N.load_data("audio")
#al, ai = N.get_audio_lstm()
#N.train_lstm("audio")

#N.get_final_model(tl, al, ti, ai)

#N.train_network()

#model = N.merged_model
#calc_test_result(model.predict([N.test_x_text, N.test_x_audio]), N.test_y, N.test_mask)
#calc_test_result(model.predict([N.train_x_text, N.train_x_audio]), N.train_y, N.train_mask)



#print(N.test_y, model.predict(N.test_x_video))

plot(s)

Loading data
Labels used for this classification: {'neutral': 0, 'sadness': 1, 'joy': 2, 'anger': 3}


FileNotFoundError: ignored

In [0]:
"""!pip uninstall tensorflow
#!pip uninstall tensorboard
#!pip install tensorboard==1.14.0 --ignore-installed
!pip install tensorflow==1.14.0 --ignore-installed

#!pip install keras-multi-head"""

Uninstalling tensorflow-2.2.0rc3:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0rc3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0rc3
     |████████████████████████████████| 109.2MB 98kB/s 
     |████████████████████████████████| 3.2MB 43.4MB/s 
     |████████████████████████████████| 1.3MB 53.6MB/s 
     |████████████████████████████████| 20.2MB 84.2MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
     |████████████████████████████████| 2.9MB 48.2MB/s 
     |████████████████████████████████| 491kB 58.7MB/s 
     |████████████████████████████████| 112kB 67.0MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |███████████